# Networks from OpenStreetMap

- import networks from OpenStreetMap.
- integrate it and run it in Flow.


In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'1.9.0'

In [1]:
# the TestEnv environment is used to simply simulate the network
from flow.envs import TestEnv

# the Experiment class is used for running simulations
from flow.core.experiment import Experiment

# all other imports are standard
from flow.core.params import VehicleParams
from flow.core.params import NetParams, SumoCarFollowingParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.core.params import SumoParams

from flow.networks import Network

## 1. Ajouter un flux de voiture

In [2]:
from flow.core.params import InFlows

inflow = InFlows()
inflow.add(veh_type="human",
           edge="4794817",
           probability= 0.3,
           depart_speed=20,
           color="green")
inflow.add(veh_type="rl",
           edge="4794817",
           probability= 0.07,
           depart_speed=20,
           color="green")
inflow.add(veh_type="human",
            edge="4783299#0",
            probability= 0.3,
            depart_lane="random",
            depart_speed=20,
            color="red")
inflow.add(veh_type="rl",
            edge="4783299#0",
            probability= 0.07,
            depart_lane="random",
            depart_speed=20,
            color="red")
inflow.add(veh_type="human",
           edge="-100822066",
           probability= 0.1,
           depart_lane= 1,  # left lane
           depart_speed= "max",
           begin= 60,  # 1 minute
           number= 30,
           color= "white")
inflow.add(veh_type="human",
            edge="155558218",
            probability= 0.2,
            depart_lane="random",
            depart_speed="max",
            color="white")
inflow.add(veh_type="rl",
            edge="155558218",
            probability= 0.05,
            depart_lane="random",
            depart_speed="max",
            color="white")

## 2. Créer des itinéraires

In [3]:
# Specifie les noms des edges du network dont les vehicules peuvent être originaire
EDGES_DISTRIBUTION = [
    "-100822066",
    "4794817",
    "4783299#0",
    "155558218",
]

In [4]:
# créer une nouvelle classe Network pour spécifier les itinéraires possibles
class IssyOSMNetwork(Network):

    def specify_routes(self, net_params):
        return {
            "-100822066": [ #N
                "-100822066",
                "-352962858#1",
                "-352962858#0",
                "-4786940#1",
                 "-4786940#0",
            ],
            
            "4794817" : [ #Loop
                "4794817",
                "4786972#0",
                "4786972#1",
                "4786972#2",
                "4786965#1",
                "4786965#2",
                "4786965#3",
                "4795729",
                "-352962858#1",
                "4795742#0",
                "4795742#1",
                "4786965#3",
                "4786965#4",
                "4786965#5",
            ],
            
            "4783299#0": [    #E
                "4783299#0",
                "4783299#1",
                "4783299#2",
                "4783299#3",
                "4783299#4",
                "4783299#5",
                "4783299#6",
                "4786940#0",
                "4786940#1",
                "352962858#0",
                "352962858#1",
                "100822066",
            ],
            
            "155558218": [
                "155558218",
                "4786940#1",
                "352962858#0",
                "352962858#1",
                "100822066",
            ],     
        }

# Personnaliser un Environnement pour le RL

More accessor objects and methods can be found within the Flow documentation at: http://berkeleyflow.readthedocs.io/en/latest/

In [ ]:
# import the base environment class
from flow.envs import Env
from gym.spaces.box import Box
from gym.spaces import Tuple
import numpy as np

## definition de la classe environnement 

In [ ]:
class myEnv(Env):
    pass

## fonction action_space
2 actions possibles pour chaque véhicule RL : +1 acceleration ou -1 acceleration

In [ ]:
class myEnv(myEnv): # update my environment class

    @property
    def action_space(self):
        num_actions = self.initial_vehicles.num_rl_vehicles
        accel_ub = self.env_params.additional_params["max_accel"]
        accel_lb = - abs(self.env_params.additional_params["max_decel"])

        return Box(low=accel_lb,
                   high=accel_ub,
                   shape=(num_actions,))

## observation_space
2 valeurs observé pour chaque véhicule: sa **position** et sa **vitesse**. En conséquence, nous avons besoin d'un espace d'observation qui est *deux fois plus grand que le nombre de véhicules* dans le network.

In [ ]:
class myEnv(myEnv):  # update my environment class

    @property
    def observation_space(self):
        return Box(
            low=0,
            high=float("inf"),
            shape=(2*self.initial_vehicles.num_vehicles,))

## apply_rl_actions
`apply_rl_actions` : transforme les commandes de l'agent RL en actions réelles du simulateur.  

Pour notre exemple, l'agent RL peut spécifier que les accélérations des véhicules RL avec la fonction **apply_acceleration**

In [ ]:
class myEnv(myEnv):  # update my environment class

    def _apply_rl_actions(self, rl_actions):
        # the names of all autonomous (RL) vehicles in the network
        rl_ids = self.k.vehicle.get_rl_ids()

        # use the base environment method to convert actions into accelerations for the rl vehicles
        self.k.vehicle.apply_acceleration(rl_ids, rl_actions)

## get_state

`get_state` : extrait des features de l'environnement et fournit ensuite des entrées à la stratégie fournie par l'agent RL. 

In [ ]:
class myEnv(myEnv):  # update my environment class

    def get_state(self, **kwargs):
        # the get_ids() method is used to get the names of all vehicles in the network
        ids = self.k.vehicle.get_ids()

        # we use the get_absolute_position method to get the positions of all vehicles
        pos = [self.k.vehicle.get_x_by_id(veh_id) for veh_id in ids]

        # we use the get_speed method to get the velocities of all vehicles
        vel = [self.k.vehicle.get_speed(veh_id) for veh_id in ids]

        # the speeds and positions are concatenated to produce the state
        return np.concatenate((pos, vel))

## compute_reward

`compute_reward` : renvoie la récompense associée à un état donné. 

Ici, la fonction de récompense est la **vitesse moyenne de tous les véhicules actuellement sur le réseau**.

In [ ]:
class myEnv(myEnv):  # update my environment class

    def compute_reward(self, rl_actions, **kwargs):
        # the get_ids() method is used to get the names of all vehicles in the network
        ids = self.k.vehicle.get_ids()

        # we next get a list of the speeds of all vehicles in the network
        speeds = self.k.vehicle.get_speed(ids)

        # finally, we return the average of all these speeds as the reward
        return np.mean(speeds)

# Testing 

In [5]:
HORIZON = 1000

In [6]:
from flow.controllers import IDMController, ContinuousRouter
from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams
from flow.core.params import VehicleParams
from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS

ADDITIONAL_ENV_PARAMS = {"max_accel": 1, "max_decel": 1}

In [7]:
from myEnv import MyEnv as myEnv

In [ ]:
from issyEnv import IssyEnv

In [8]:
from flow.controllers import IDMController, ContinuousRouter, RLController
from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams
from flow.core.params import VehicleParams
from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS

ADDITIONAL_ENV_PARAMS = {"max_accel": 1, "max_decel": 1, "beta" : 55}

# SUMO PARAM
sumo_params = SumoParams(sim_step=0.1, render=True)

# create VEHICLE
vehicles = VehicleParams()
vehicles.add("human",acceleration_controller=(IDMController, {}), num_vehicles=20)
vehicles.add("rl",acceleration_controller=(RLController, {}), num_vehicles=1)
#vehicles.add(veh_id="human",
 #            acceleration_controller=(IDMController, {}),
  #           num_vehicles=22)

# ENVIRONMENT PARAM
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS, horizon=HORIZON)

# NETWORK PARAM
additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params, inflows=inflow, osm_path='issy.osm')

# NETWORK
network = IssyOSMNetwork

# INITIAL CONFIG
initial_config = InitialConfig(edges_distribution=EDGES_DISTRIBUTION) #spacing="random",

# dictionnaire FLOW_PARAM
flow_params = dict( exp_tag  = 'ISSY_RL_test',
                    env_name = myEnv,  # using my new environment for the simulation
                    network  = network,
                    simulator='traci',
                    sim      = sumo_params,
                    env      = env_params,
                    net      = net_params,
                    veh      = vehicles,
                    initial  = initial_config)

# create EXPERIMENT with class created
exp = Experiment(flow_params)

# RUN SIMULATION SUMO
_ = exp.run(1)

 Retrying in 1 seconds
**********************************************************
**********************************************************
**********************************************************
significantly decrease after large number of rollouts. In 
order to avoid this, set SumoParams(restart_instance=True).
**********************************************************
**********************************************************
**********************************************************
Round 0, return: 6655.56035050818
Average, std returns: 6655.56035050818, 0.0
Average, std velocities: 6.65556035050819, 0.0
Average, std outflows: 468.0, 0.0
Total time: 59.85075616836548
steps/second: 23.05016674672746


# Training

Pour qu'un environnement puisse être entrainé, l'environnement doit être accessible via l'importation à partir de flow.envs. <font color='red'> On copie alors l'environnement créé dans un fichier .py et on importe l'environnement dans `flow.envs.__init__.py`. </font> 

In [ ]:
# NOTE: only runs if the above procedure have been performed
from flow.envs import myEnv

In [7]:
from myEnv import MyEnv as myEnv

In [8]:
from issyEnv import IssyEnv

In [9]:
from IssyExperiment import IssyExperiment, IssyExperimentParams, RayClusterParams

In [8]:
import json
import ray
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

# from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import VehicleParams, SumoCarFollowingParams
from flow.controllers import RLController, IDMController, ContinuousRouter


# number of rollouts per training iteration
N_ROLLOUTS = 20
# number of parallel workers
N_CPUS = 2


# SUMO PARAM
sumo_params = SumoParams(sim_step=0.1, render=False, restart_instance=True)

# create VEHICLE : 1 RL and 21 humans (simulated comportement)
vehicles = VehicleParams()
vehicles.add(veh_id="rl", acceleration_controller=(RLController, {}), num_vehicles=1)
vehicles.add(veh_id="human", acceleration_controller=(IDMController, {}), num_vehicles=21)

# ENVIRONMENT PARAM
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS, horizon=HORIZON)

# NETWORK PARAM
additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params, inflows=inflow, osm_path='/app/notebooks/networks/issy.osm')

# NETWORK
network = IssyOSMNetwork

# INITIAL CONFIG
initial_config = InitialConfig(edges_distribution=EDGES_DISTRIBUTION)

flow_params = dict( exp_tag   = "ISSY_RL_train", 
                    env_name  = myEnv,  
                    network   = IssyOSMNetwork,
                    simulator = 'traci', # simulator that is used by the experiment
                    sim       = sumo_params,
                    env       = env_params,
                    net       = net_params,
                    veh       = vehicles,
                    initial   = initial_config)

def setup_exps():
    """Return the relevant components of an RLlib experiment.

    Returns
    -------
    str
        name of the training algorithm
    str
        name of the gym environment to be trained
    dict
        training configuration parameters
    """
    alg_run = "PPO"
    agent_cls = get_agent_class(alg_run)
    config = agent_cls._default_config.copy()
    config["num_workers"] = N_CPUS
    config["train_batch_size"] = HORIZON * N_ROLLOUTS
    config["gamma"] = 0.999  # discount rate
    config["model"].update({"fcnet_hiddens": [3, 3]})
    config["use_gae"] = True
    config["lambda"] = 0.97
    config["kl_target"] = 0.02
    config["num_sgd_iter"] = 10
    config['clip_actions'] = False  # FIXME(ev) temporary ray bug
    config["horizon"] = HORIZON

    # save the flow params for replay
    flow_json = json.dumps( flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    
    return alg_run, gym_name, config


alg_run, gym_name, config = setup_exps()

ray.init(num_cpus=N_CPUS + 1, object_store_memory=1000000000)

trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 20,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 4,
        },
    }
})

2020-03-25 23:55:42,640	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-03-25_23-55-42_639689_2271/logs.
2020-03-25 23:55:42,758	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:64538 to respond...
2020-03-25 23:55:42,904	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:14789 to respond...
2020-03-25 23:55:42,911	INFO services.py:809 -- Starting Redis shard with 0.21 GB max memory.
2020-03-25 23:55:42,980	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-03-25_23-55-42_639689_2271/logs.
2020-03-25 23:55:42,986	WARNING services.py:1352 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the f

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/0 GPUs
Memory usage on this node: 0.5/1.0 GB



2020-03-25 23:55:43,549	WARNING util.py:145 -- The `start_trial` operation took 0.27845144271850586 seconds to complete, which may be a performance bottleneck.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/0 GPUs
Memory usage on this node: 0.5/1.0 GB
Result logdir: /root/ray_results/ISSY_RL_train
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - PPO_MyEnv-v0_0:	RUNNING

(pid=2306) 2020-03-25 23:55:50,337	WARNING ppo.py:143 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
(pid=2306) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.residential' (first occurence for edge '4786965#0').
(pid=2306) Warning: Discarding unknown compound 'cycleway.opposite_track' in type 'cycleway.opposite_track|highway.residential' (first occurence for edge '4786972#0').
(pid=2306) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.living_street' (first occurence for edge '4795758').
(pid=2306) Warning: Discarding unknown compound 'cycleway.lane' in type 'cycleway.lane|highway.secondar

(pid=2360) Success.
(pid=2364) Success.
(pid=2360) 2020-03-25 23:56:21,813	INFO rollout_worker.py:319 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=2364) 2020-03-25 23:56:21,814	INFO rollout_worker.py:319 -- Creating policy evaluation worker 2 on CPU (please ignore any CUDA init errors)
(pid=2360) 2020-03-25 23:56:22.166339: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2
(pid=2364) 2020-03-25 23:56:22.166672: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2
(pid=2360) /opt/conda/envs/flow/lib/python3.6/site-packages/ray/rllib/models/tf/misc.py:14: RuntimeWarning: divide by zero encountered in true_divide
(pid=2360)   out *= std / np.sqrt(np.square(out).sum(axis=0, keepdims=True))
(pid=2364) /opt/conda/envs/flow/lib/python3.6

...

(pid=2470) 2020-03-25 23:57:07.438874: E tensorflow/core/common_runtime/bfc_allocator.cc:246] tried to allocate 0 bytes
(pid=2470) 2020-03-25 23:57:07.438926: W tensorflow/core/common_runtime/allocator_retry.cc:32] Request to allocate 0 bytes
(pid=2470) 2020-03-25 23:57:07.438953: E tensorflow/core/common_runtime/bfc_allocator.cc:381] tried to deallocate nullptr
(pid=2515) Warning: Removed a road without junctions: -21549451
(pid=2515) Warning: Removed a road without junctions: 21549451
(pid=2515) Warning: The traffic light '30624291' does not control any links; it will not be build.
(pid=2515) Warning: Could not build program '0' for traffic light '30624291'
(pid=2515) Warning: Minor green from edge '-352962858#0' to edge '4786972#0' exceeds 19.44m/s. Maybe a left-turn lane is missing.
(pid=2515) Warning: Minor green from edge '352962858#0' to edge '4794820' exceeds 19.44m/s. Maybe a left-turn lane is missing.
(pid=2515) Warning: Minor green from edge '-352962858#1' to edge '4795742#0

(pid=2515) 2020-03-25 23:57:13.747013: E tensorflow/core/common_runtime/bfc_allocator.cc:246] tried to allocate 0 bytes
(pid=2515) 2020-03-25 23:57:13.747066: W tensorflow/core/common_runtime/allocator_retry.cc:32] Request to allocate 0 bytes
(pid=2515) 2020-03-25 23:57:13.831337: E tensorflow/core/common_runtime/bfc_allocator.cc:246] tried to allocate 0 bytes
(pid=2515) 2020-03-25 23:57:13.831430: W tensorflow/core/common_runtime/allocator_retry.cc:32] Request to allocate 0 bytes
(pid=2515) 2020-03-25 23:57:13.831453: E tensorflow/core/common_runtime/bfc_allocator.cc:246] tried to allocate 0 bytes
(pid=2515) 2020-03-25 23:57:13.831470: W tensorflow/core/common_runtime/allocator_retry.cc:32] Request to allocate 0 bytes
(pid=2515) 2020-03-25 23:57:13.831486: E tensorflow/core/common_runtime/bfc_allocator.cc:246] tried to allocate 0 bytes
(pid=2515) 2020-03-25 23:57:13.831502: W tensorflow/core/common_runtime/allocator_retry.cc:32] Request to allocate 0 bytes
(pid=2515) 2020-03-25 23:57:

2020-03-25 23:57:14,467	ERROR worker.py:1716 -- listen_error_messages_raylet: Error 111 connecting to 172.17.0.2:64538. Connection refused.
2020-03-25 23:57:14,471	ERROR worker.py:1616 -- print_logs: Error 111 connecting to 172.17.0.2:64538. Connection refused.
2020-03-25 23:57:14,471	ERROR import_thread.py:89 -- ImportThread: Error 111 connecting to 172.17.0.2:64538. Connection refused.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/envs/flow/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-15f1be152a85>", line 105, in <module>
    "training_iteration": 4,
  File "/opt/conda/envs/flow/lib/python3.6/site-packages/ray/tune/tune.py", line 324, in run_experiments
    return_trials=True)
  File "/opt/conda/envs/flow/lib/python3.6/site-packages/ray/tune/tune.py", line 244, in run
    runner.step()
  File "/opt/conda/envs/flow/lib/python3.6/site-packages/ray/tune/trial_runner.py", line 328, in step
    self._process_events()  # blocking
  File "/opt/conda/envs/flow/lib/python3.6/site-packages/ray/tune/trial_runner.py", line 492, in _process_events
    trial = self.trial_executor.get_next_available_trial()  # blocking
  File "/opt/conda/envs/flow/lib/python3.6/site-packages/ray/tune/ray_trial_executor.py", line 318, in get_next_available_trial
    [result

KeyboardInterrupt: 

# /!\ Voir tutoriel 10 pour controle des feux